# Constructing linear model for OER adsorption energies
---

TODO:
  * Add bulk formation energy as descriptor

# Import Modules

In [ ]:
import os
print(os.getcwd())
import sys

import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd

# pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.options.display.max_colwidth = 100

import plotly.graph_objs as go

# #########################################################
from methods import (
    get_df_eff_ox,
    get_df_ads,
    get_df_features,
    get_df_jobs,
    )

# #########################################################
from layout import layout

# Script Inputs

In [ ]:
verbose = True
# verbose = False

# Read Data

In [ ]:
df_eff_ox = get_df_eff_ox()

df_ads = get_df_ads()

df_features = get_df_features()

df_jobs = get_df_jobs()

In [ ]:
feature_cols = df_features.columns.tolist()

In [ ]:
data_dict_list = []
for i_cnt, row_i in df_ads.iterrows():
    data_dict_i = dict()

    # #####################################################
    compenv_i = row_i.compenv
    slab_id_i = row_i.slab_id
    active_site_i = row_i.active_site
    g_o_i = row_i.g_o
    job_id_o_i = row_i.job_id_o
    # active_site_i = row_i.active_site
    # #####################################################

    # #########################################################
    row_jobs_i = df_jobs.loc[job_id_o_i]
    # #########################################################
    att_num_i = row_jobs_i.att_num
    # #########################################################

    # #####################################################
    data_dict_i["compenv"] = compenv_i
    data_dict_i["slab_id"] = slab_id_i
    data_dict_i["ads"] = "o"
    data_dict_i["active_site"] = active_site_i
    data_dict_i["att_num"] = att_num_i
    data_dict_i["g_o"] = g_o_i
    data_dict_i["job_id_o"] = job_id_o_i
    data_dict_i["active_site"] = active_site_i
    # #####################################################
    data_dict_list.append(data_dict_i)
    # #####################################################

# #########################################################
df = pd.DataFrame(data_dict_list)
df = df.dropna()
df = df.set_index(["compenv", "slab_id", "ads", "active_site", "att_num"], drop=False)

In [ ]:
# compenv	slab_id	ads	active_site	att_num	g_o	job_id_o
df.iloc[0:1]

# Combine features dataframe with adsorption energy

In [ ]:
df_tmp = pd.concat([df, df_features], axis=1)

df_tmp = df_tmp.dropna()

df_tmp = df_tmp.drop(columns=["compenv", "slab_id", "ads", "active_site", "att_num", ])

In [ ]:
assert False

In [ ]:
for feature_i in feature_cols:
    mean_val = df_tmp[feature_i].mean()

    std_val = df_tmp[feature_i].std()

    df_tmp[feature_i + "_stan"] = (df_tmp[feature_i] - mean_val) / std_val

In [ ]:
# for name_i, row_i in df_tmp.iterrows():

#     # #########################################################
#     compenv_i = name_i[0]
#     slab_id_i = name_i[1]
#     ads_i = name_i[2]
#     active_site_i = name_i[3]
#     att_num_i = name_i[4]
#     # #########################################################

#     job_id_o_i = row_i.job_id_o

#     name_i = job_id_o_i + "__" + str(int(active_site_i)).zfill(3)

In [ ]:
def method(row_i):
    # #########################################################
    name_i = row_i.name
    # #########################################################
    compenv_i = name_i[0]
    slab_id_i = name_i[1]
    ads_i = name_i[2]
    active_site_i = name_i[3]
    att_num_i = name_i[4]
    # #########################################################
    
    job_id_o_i = row_i.job_id_o

    name_i = job_id_o_i + "__" + str(int(active_site_i)).zfill(3)
    
    return(name_i)

# df_i = df_tmp
df_tmp["name_str"] = df_tmp.apply(
    method,
    axis=1)
# df_tmp = df_i

In [ ]:
feature_cols_stan = [i + "_stan" for i in feature_cols]
feature_cols_stan

In [ ]:
# X = df_tmp[feature_cols].to_numpy().reshape(-1, len(feature_cols))
X = df_tmp[feature_cols_stan].to_numpy().reshape(-1, len(feature_cols))
y = df_tmp.g_o

model = LinearRegression()
model.fit(X, y)

y_predict = model.predict(X)

# Plotting

In [ ]:
data = []

In [ ]:
# x_parity = y_parity = np.linspace(0, 5.5, num=100, )
x_parity = y_parity = np.linspace(0., 8., num=100, )

trace_i = go.Scatter(
    x=x_parity,
    y=y_parity,
    line=go.scatter.Line(color="black", width=2.),
    mode="lines")
data.append(trace_i)

In [ ]:
trace_i = go.Scatter(
    y=y,
    x=y_predict,
    mode="markers",
    marker=go.scatter.Marker(
        size=12,
        ),
    # mode="markers+text",
    # name="Markers and Text",
    text=df_tmp.name_str,
    textposition="bottom center",
    )
data.append(trace_i)

In [ ]:
# go.Scatter?

# plotly.graph_objects.scatter.Marker
# go.scatter.Marker?

In [ ]:
layout.xaxis.title.text = "Predicted ΔG<sub>*O</sub>"

layout.yaxis.title.text = "Simulated ΔG<sub>*O</sub>"


layout.xaxis.title.font.size = 25
layout.yaxis.title.font.size = 25

layout.yaxis.tickfont.size = 20
layout.xaxis.tickfont.size = 20

layout.xaxis.range = [2.5, 5.5]

layout.showlegend = False

# layout.xaxis.scaleanchor = "y"

# go.layout.XAxis?

In [ ]:
dd = 0.2
layout.xaxis.range = [
    np.min(y_predict) - dd,
    np.max(y_predict) + dd,
    ]


layout.yaxis.range = [
    np.min(y) - dd,
    np.max(y) + dd,
    ]

In [ ]:
# np.min(y)
# np.max(y)

In [ ]:
fig = go.Figure(data=data, layout=layout)

from plotting.my_plotly import my_plotly_plot

my_plotly_plot(
    figure=fig,
    plot_name="parity_plot",
    write_html=True)

fig.show()

In [ ]:
print("model.score(X, y):", model.score(X, y))
print("")

# print(feature_cols)
# print(model.coef_)

for i, j in zip(feature_cols, model.coef_):
    print(i, ": ", j, sep="")

In [ ]:
fragments = [
    "faw",
    ]

for name_i, row_i in df_tmp.iterrows():
    # tmp = 42
    name_str_i = row_i.name_str
    for frag_i in fragments:

        if frag_i in name_str_i:
            print(name_str_i)
            # print("IYUIHUuids")

# name_str_i